In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path
import plotly.graph_objects as go
import re

from glitch.impedance import EISSpectrumDoc, SingleSpectrum

data_folder = Path("/Users/andreakowal/Downloads/MTRON/DI Water").expanduser()

# background correction files
OC_path = Path("/Users/andreakowal/Downloads/MTRON/02_07_25_OCTest_C01.mpt").expanduser()
SC_path = Path("/Users/andreakowal/Downloads/MTRON/02_03_25_SCTest_C01.mpt").expanduser()
oc = EISSpectrumDoc.from_eclab_mpt(OC_path)
sc = EISSpectrumDoc.from_eclab_mpt(SC_path)

# sort membrane files by membrane ID and replicate number
def extract_membrane_and_replicate (data_folder):
    membrane_ID = re.search(r"_(\d+)_R", data_folder.name) #Membrane number is after the first underscore
    replicate = re.search(r"_R(\d+)_", data_folder.name) #Replicate number is after the R and before 3rd underscore

    membrane_number = int(membrane_ID.group(1)) if membrane_ID else "Membrane ID not found"
    replicate_number = int(replicate.group(1)) if replicate else "Replicate not found"
    return (membrane_number, replicate_number)

nyquist_results = []

def trendline_intercept(Z_real, Z_imag):
    slope, intercept = np.polyfit(Z_real, Z_imag, 1)
    return -intercept / slope

for file_path in sorted(data_folder.glob("*.mpt"), key = extract_membrane_and_replicate):
    print(f"Processing {file_path.name}")

    my_spectrum = EISSpectrumDoc.from_eclab_mpt(file_path)
    my_spectrum.background_correct(oc.cycles_raw[0], sc.cycles_raw[0])

    raw_data = my_spectrum.cycles_raw[0]
    mask = raw_data.frequencies <= 250_000
    freqs_filtered = raw_data.frequencies[mask]
    impedance_filtered = raw_data.impedance[mask]

    Z_real = impedance_filtered.real
    Z_imag = impedance_filtered.imag

    df_points = pd.DataFrame({
        "Frequency (Hz)": freqs_filtered,
        "Z_real (Ohm)": Z_real,
        "Z_imag (Ohm)": Z_imag
    })
    output_filename = f"{file_path.stem}_points.csv"
    df_points.to_csv(output_filename, index=False)
    print(f"Saved data table to: {output_filename}")

    # Create trendline
    trendline = np.poly1d(np.polyfit(Z_real, Z_imag, 1))
    Z_imag_trend = trendline(Z_real)

    # Plotly Nyquist plot
    fig = go.Figure()

    fig.add_trace(go.Scatter(
        x=Z_real,
        y=Z_imag, 
        mode='markers',
        name='EIS Data',
        marker=dict(color='blue')
    ))

    fig.add_trace(go.Scatter(
        x=Z_real,
        y=Z_imag_trend,
        mode='lines',
        name='Trendline',
        line=dict(color='black')
    ))

    fig.update_layout(
        title=file_path.stem,
        xaxis_title='Re{Z} (Ohm)',
        yaxis_title='-Im{Z} (Ohm)',
        xaxis=dict(scaleanchor="y", scaleratio=1),  # equal aspect ratio
        yaxis=dict(),
        width=700,
        height=600
    )

    fig.show()

    filtered_cycle = SingleSpectrum(frequencies=freqs_filtered, impedance=impedance_filtered)
    filtered_doc = EISSpectrumDoc(cycles=[filtered_cycle])

    x_intercept = trendline_intercept(Z_real, Z_imag)

    nyquist_results.append({"filename": file_path.name, "real intercept": x_intercept})

df = pd.DataFrame(nyquist_results)
df.to_csv("EIS_summary.csv", index=False)
print(df)



Processing DIWater_01_R1_FKS50_20250421_C01.mpt
Saved data table to: DIWater_01_R1_FKS50_20250421_C01_points.csv


Processing DIWater_01_R2_FKS50_20250421_C01.mpt
Saved data table to: DIWater_01_R2_FKS50_20250421_C01_points.csv


Processing DIWater_01_R3_FKS50_20250421_C01.mpt
Saved data table to: DIWater_01_R3_FKS50_20250421_C01_points.csv


Processing DIWater_02_R1_FKS50_20250421_C01.mpt
Saved data table to: DIWater_02_R1_FKS50_20250421_C01_points.csv


Processing DIWater_02_R2_FKS50_20250421_C01.mpt
Saved data table to: DIWater_02_R2_FKS50_20250421_C01_points.csv


Processing DIWater_02_R3_FKS50_20250421_C01.mpt
Saved data table to: DIWater_02_R3_FKS50_20250421_C01_points.csv


Processing DIWater_03_R1_FKS50_20250421_C01.mpt
Saved data table to: DIWater_03_R1_FKS50_20250421_C01_points.csv


Processing DIWater_03_R2_FKS50_20250421_C01.mpt
Saved data table to: DIWater_03_R2_FKS50_20250421_C01_points.csv


Processing DIWater_03_R3_FKS50_20250421_C01.mpt
Saved data table to: DIWater_03_R3_FKS50_20250421_C01_points.csv


                               filename  real intercept
0  DIWater_01_R1_FKS50_20250421_C01.mpt        1.203503
1  DIWater_01_R2_FKS50_20250421_C01.mpt        2.881363
2  DIWater_01_R3_FKS50_20250421_C01.mpt        3.279020
3  DIWater_02_R1_FKS50_20250421_C01.mpt       -0.698977
4  DIWater_02_R2_FKS50_20250421_C01.mpt        1.803997
5  DIWater_02_R3_FKS50_20250421_C01.mpt        2.290044
6  DIWater_03_R1_FKS50_20250421_C01.mpt        1.056807
7  DIWater_03_R2_FKS50_20250421_C01.mpt        1.910688
8  DIWater_03_R3_FKS50_20250421_C01.mpt        5.309363
